In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import math
from transformer_lens import HookedTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
def normalize(R):
    with torch.no_grad():
        mean = R.mean(dim=0)
        R = R - mean
        norms = torch.norm(R, p=2, dim=1, keepdim=True)
        R = R/norms
    return R

def cal_cov(R):
    with torch.no_grad():
        Z = torch.nn.functional.normalize(R, dim=1)
        A = torch.matmul(Z.T, Z)/Z.shape[0]
    return A

def cal_entropy(A):
    with torch.no_grad():
        eig_val = torch.svd(A / torch.trace(A))[1] 
        entropy = - (eig_val * torch.log(eig_val)).nansum().item()
        normalized_entropy = entropy/math.log(A.shape[0])
    return normalized_entropy


In [6]:
#text = " The threee littile pigs is a beloved folk tale that tells the story of three pigs, each building a house out of different materials: straw, sticks, and bricks. The youngest pig opts for straw, the middle pig chooses sticks, while the eldest builds his house out of bricks. When a big bad wolf comes to eat them, he easily destroys the first two houses by huffing and puffing, devouring the first two pigs. However, the third pig's brick house withstands the wolf's attempts. Frustrated, the wolf tries to enter through the chimney, but the clever pig boils a pot of water just in time, ending the wolf's threat once and for all. This classic tale highlights the virtues of hard work and resourcefulness, as the most diligent and smartest pig not only survives but thrives."

In [ ]:
# model: HookedTransformer = HookedTransformer.from_pretrained("tiny-stories-3M")
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2")
prompt: str = "Once upon a time, there was a hunter. He lived in a small house near the woods. He had a dog named Spot."


In [15]:
model_path = "gpt2" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

text = "good job"
inputs = tokenizer(text, return_tensors="pt").to('cpu')
with torch.no_grad():
    R = model(inputs.input_ids)[0][0, :, :]
    R = normalize(R)
    A = cal_cov(R)
    Entropy = cal_entropy(A)
print(Entropy)

0.0001189069367975216


In [11]:
model(inputs.input_ids)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.0796, -0.0654, -0.0842,  ..., -0.1442, -0.0456,  0.0143],
         [-0.0426, -0.3873, -0.3798,  ..., -0.1628,  0.2764,  0.2956],
         [ 0.1341, -0.1601,  0.5899,  ..., -0.3328,  0.1413,  0.1053],
         ...,
         [-0.0718, -0.6685, -0.1171,  ..., -0.2038, -0.0289, -0.1591],
         [-0.6112, -0.3013, -0.6746,  ..., -0.0257,  0.2878, -0.0120],
         [ 0.1163, -0.5331, -0.0777,  ..., -0.2188, -0.1378, -0.2807]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.5577e+00,  2.0585e+00,  1.3060e+00,  ..., -1.3825e+00,
           -6.3336e-01,  1.2624e+00],
          [-2.5532e+00,  2.6298e+00,  1.7484e+00,  ..., -8.9923e-01,
           -1.2752e+00,  1.8641e+00],
          [-1.8864e+00,  1.5099e+00,  1.2464e+00,  ..., -1.0845e-02,
           -1.1708e+00,  1.6294e+00],
          ...,
          [-2.9377e+00,  2.3325e+00,  2.1564e+00,  ..., -8.5890e-01,
           -1.6372e+00,  1.9465e+00],
    